# Usage of `src.models.weather` module.

This notebook outlies the basic usage of the `src.models.weather` module. Used to define weather models and estimate values at different altitudes
 
**Requirements**
 - A csv with weather station data

**Helpful Links**
 - [Weather Models Wiki](https://github.com/Flight-Path-Analysis/FlightPathAnalysis/wiki/Weather_Models)


## Basic Setup

In [1]:
import sys
# This variable should indicate the path from this Jupyter Notebook to the root directory of the repo.
root_path = '../'
# Adds the repo's root to the list of paths
sys.path.append(root_path)

# Package to read yml files
import yaml
# Package to handle file paths
import os
# Package to deal with DataFrames
import pandas as pd
# Package to plot stuff
import matplotlib.pyplot as plt
import numpy as np

# Function to clear output from jupyter notebook
from IPython.display import clear_output
# Package for compressing dataframes into file
from src.models import weather
# Utilities package
from src.common import utils

root_path = os.path.normpath(root_path) # Path from this notebook to the root directory
config_path_from_root = os.path.normpath('config/config_tutorial.yml') # Path from root to the desired config file
config_path = os.path.join(root_path, config_path_from_root) # Defining path from this notebook to config file

# Loading config file
with open(config_path, 'r',  encoding='utf8') as file:
    config = yaml.safe_load(file)

# Defining "clear-output" function to feed into logger
def clear():
    clear_output(wait=True)

# Creates an instance of a logger class to log all that happens, optional (but encouraged).
logger = utils.Logger(config, clear_function=clear)

# Loading weather station data
data_directory = 'tutorial_data'
station_data = pd.read_csv(os.path.join(data_directory, 'station_data.csv'), index_col = 0)

station_data

,timestamp,lon,lat,tmpf,relh,drct,sknt,p01i,skyc4,skyl1,...,smps_N,tmpc,elevation,sigma,station,valid,skyc1,skyc2,skyc3,metar
0,1672444800,-176.646,51.878,35.600000,93.080000,350.0,9.0,0.0,NaN,1600.0,...,4.559656,2.000000,4.0,585663.326384,PADK,2022-12-31 00:00,SCT,BKN,BKN,PADK 310000Z AUTO 35009KT SCT016 BKN023 BKN029...
1,1672445100,-176.646,51.878,35.600000,93.080000,350.0,8.0,0.0,NaN,1300.0,...,4.053028,2.000000,4.0,585663.326384,PADK,2022-12-31 00:05,SCT,SCT,BKN,PADK 310005Z AUTO 35008KT SCT013 SCT018 BKN025...
2,1672445400,-176.646,51.878,35.600000,93.080000,360.0,9.0,0.0,NaN,1500.0,...,4.629996,2.000000,4.0,585663.326384,PADK,2022-12-31 00:10,BKN,BKN,BKN,PADK 310010Z AUTO 36009KT BKN015 BKN020 BKN034...
3,1672445700,-176.646,51.878,35.600000,93.080000,360.0,9.0,0.0,NaN,1500.0,...,4.629996,2.000000,4.0,585663.326384,PADK,2022-12-31 00:15,BKN,BKN,NaN,PADK 310015Z AUTO 36009KT BKN015 BKN034 02/01 ...
4,1672446000,-176.646,51.878,35.600000,93.080000,350.0,9.0,0.0,NaN,1200.0,...,4.559656,2.000000,4.0,585663.326384,PADK,2022-12-31 00:20,FEW,SCT,BKN,PADK 310020Z AUTO 35009KT FEW012 SCT017 BKN034...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,1672530000,-176.646,51.878,35.869388,92.588367,360.0,5.0,0.0,NaN,1100.0,...,2.572220,2.149660,4.0,585663.326384,PADK,2022-12-31 23:40,FEW,BKN,BKN,PADK 312340Z AUTO 36005KT FEW011 BKN019 BKN025...
326,1672530300,-176.646,51.878,35.910204,92.513878,10.0,4.0,0.0,NaN,1100.0,...,2.026514,2.172336,4.0,585663.326384,PADK,2022-12-31 23:45,FEW,SCT,BKN,PADK 312345Z AUTO 01004KT FEW011 SCT016 BKN021...
327,1672530600,-176.646,51.878,35.951020,92.439388,360.0,7.0,0.0,NaN,1100.0,...,3.601108,2.195011,4.0,585663.326384,PADK,2022-12-31 23:50,FEW,BKN,BKN,PADK 312350Z AUTO 36007KT FEW011 BKN016 BKN021...
328,1672530900,-176.646,51.878,35.991837,92.364898,350.0,7.0,0.0,NaN,1100.0,...,3.546399,2.217687,4.0,585663.326384,PADK,2022-12-31 23:55,FEW,BKN,BKN,PADK 312355Z AUTO 35007KT FEW011 BKN016 BKN021...


## Calibrate Weather Stations Weather Models

This code creates an instance of each of the available weather models for each row in the station data.
The calibration of the weather model for one time is completely independent of the data or weather models at any other time.

The currently available weather models are:
 - Temperature
   - The temperature model is a simple Lapse Rate model, that is, the temperature decreases linearly with height.
 - Wind Model
   - The wind model is again a simple Lapse Rate model, but with increasing wind with height. This model is not used since it's not a reliable measure, and we can estimate the wind component in the direction of aicrafts from aicraft data.
   - The East and North components of the wind vector are modeled separately
 - Air Pressure
   - The air pressure model is estimated from the Temperature model, where:
$$P(h) = A exp\left(-\frac{g M}{R}\int_0^h\frac{1}{T(h')}dh'\right)$$
   - The precision and range of the numerical integral are specified by `config['models']['numerical']['integration-precision']` and `config['models']['numerical']['max-height']`.
   - All the other constants can be found on the `models` seciton in the config file
 - Air Density
   - The air density model is estimated from the temperature and pressure models simply by:
   $$\rho(h) = \frac{P(h)}{R_d\times T(h)}$$
 - Clouds
   - The cloud coverage model is an interpolation of the sky condition as reported by the `skycX` and `skylX` variables. These are mapped to a number between 0 and 1 and interpolated.
   - Extrapolations are clipped to the nearest valid entry.
   - If no valid entry, return 0

In [2]:
station_data = weather.calibrate_stations(station_data, config)

station_data

Index(['timestamp', 'lon', 'lat', 'tmpf', 'relh', 'drct', 'sknt', 'p01i',
       'skyc4', 'skyl1', 'skyl2', 'skyl3', 'skyl4', 'wxcodes',
       'ice_accretion_1hr', 'smps', 'sknt_E', 'sknt_N', 'smps_E', 'smps_N',
       'tmpc', 'elevation', 'sigma', 'station', 'valid', 'skyc1', 'skyc2',
       'skyc3', 'metar', 'tmpf_model', 'sknt_E_model', 'sknt_N_model',
       'air_pressure_model', 'air_density_model', 'cloud_model',
       'tmpf_sea_level', 'sknt_E_sea_level', 'sknt_N_sea_level',
       'air_pressure_sea_level', 'air_density_sea_level', 'cloud_sea_level'],
      dtype='object')

## Predicting weather conditions at a given height

In [6]:
height = 1000
quantities = {'tmpf':[], 'sknt_N':[], 'sknt_E':[], 'air_pressure':[], 'air_density':[], 'cloud':[]}

for i, quantity in enumerate(quantities):
    for _, row in station_data.iterrows():
        quantities[quantity] += [row[f'{quantity}_model'].predict([height])[0]]
    station_data[f'{quantity}_at_{height}'] = quantities[quantity]
station_data

,timestamp,lon,lat,tmpf,relh,drct,sknt,p01i,skyc4,skyl1,...,sknt_N_sea_level,air_pressure_sea_level,air_density_sea_level,cloud_sea_level,tmpf_at_1000,sknt_N_at_1000,sknt_E_at_1000,air_pressure_at_1000,air_density_at_1000,cloud_at_1000
0,1672444800,-176.646,51.878,35.600000,93.080000,350.0,9.0,0.0,NaN,1600.0,...,-1.642834,1013.25,0.012828,0.4,23.946800,28.783270,18.357166,893.445611,0.011585,0.4
1,1672445100,-176.646,51.878,35.600000,93.080000,350.0,8.0,0.0,NaN,1300.0,...,-1.469185,1013.25,0.012828,0.4,23.946800,27.798462,18.530815,893.445611,0.011585,0.4
2,1672445400,-176.646,51.878,35.600000,93.080000,360.0,9.0,0.0,NaN,1500.0,...,-0.080000,1013.25,0.012828,0.6,23.946800,28.920000,19.920000,893.445611,0.011585,0.6
3,1672445700,-176.646,51.878,35.600000,93.080000,360.0,9.0,0.0,NaN,1500.0,...,-0.080000,1013.25,0.012828,0.6,23.946800,28.920000,19.920000,893.445611,0.011585,0.6
4,1672446000,-176.646,51.878,35.600000,93.080000,350.0,9.0,0.0,NaN,1200.0,...,-1.642834,1013.25,0.012828,0.2,23.946800,28.783270,18.357166,893.445611,0.011585,0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,1672530000,-176.646,51.878,35.869388,92.588367,360.0,5.0,0.0,NaN,1100.0,...,-0.080000,1013.25,0.012821,0.2,24.216188,24.920000,19.920000,893.507461,0.011579,0.2
326,1672530300,-176.646,51.878,35.910204,92.513878,10.0,4.0,0.0,NaN,1100.0,...,0.614593,1013.25,0.012820,0.2,24.257004,23.859231,20.614593,893.516827,0.011578,0.2
327,1672530600,-176.646,51.878,35.951020,92.439388,360.0,7.0,0.0,NaN,1100.0,...,-0.080000,1013.25,0.012819,0.2,24.297820,26.920000,19.920000,893.526191,0.011577,0.2
328,1672530900,-176.646,51.878,35.991837,92.364898,350.0,7.0,0.0,NaN,1100.0,...,-1.295537,1013.25,0.012818,0.2,24.338637,26.813654,18.704463,893.535554,0.011576,0.2
